# Sección 1: Regresión por GPU

## Carga y tratamiento de datos

In [ ]:
import os
import os.path

import cudf
import cupy as cp
import cuml
from cuml.metrics.regression import mean_squared_error as mnsq

import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import mean_squared_error as mnsq_cpu

%run ../utils/f_northing.py
%run ../utils/f_northing_numpy.py
%run ../utils/f_price_range.py
%run ../utils/f_static_data.py
%run ../utils/f_utils.py

cities_to_use = ['sevilla']
#cities_to_use = ['shanghai']
#cities_to_use = cities_to_use_1()
#cities_to_use = cities_to_use_2()
columns_to_use = columns_to_use()
columns_to_fit = columns_to_fit()

cuml.set_global_output_type('cudf')

In [ ]:
%%time
listings = cudf.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df = cudf.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if(temp_df['host_total_listings_count'].dtype != 'int32'):
                    temp_df['host_total_listings_count'] = temp_df['host_total_listings_count'].astype('int32').fillna(-1)
                if(temp_df['bathrooms'].dtype != 'int32'):
                    temp_df['bathrooms'] = temp_df['bathrooms'].astype('int32').fillna(-1)
                if(temp_df['bedrooms'].dtype != 'int32'):
                    temp_df['bedrooms'] = temp_df['bedrooms'].astype('int32').fillna(-1)
                if(temp_df['beds'].dtype != 'int32'):
                    temp_df['beds'] = temp_df['beds'].astype('int32').fillna(-1)
                if listings.size == 0:
                    listings = temp_df
                else:
                    for column in listings.columns:
                        if listings[column].dtype != temp_df[column].dtype:
                            print('Found error: '+column+' type '+listings[column].dtype.name+' doesnt match '+temp_df[column].dtype.name)
                    listings = listings.append(temp_df)
                    
listings = listings.drop_duplicates().reset_index(drop=True)

type_conversion(listings, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month', 'minimum_nights', 'maximum_nights', 'availability_30', 'availability_90', 'availability_365', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'])
column_factorize(listings, ['neighbourhood_cleansed', 'host_response_time', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'property_type', 'room_type', 'instant_bookable'])

clean_format_strings(listings, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price(listings, ['price'])

cupy_lat = cp.asarray(listings['latitude'])
cupy_long = cp.asarray(listings['longitude'])
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
listings['northing'] = cudf.Series(n_cupy_array).astype('float32')
listings['easting'] = cudf.Series(e_cupy_array).astype('float32')

listings.head()

## Aplicación de modelo de regresión para predicción de valores

In [ ]:
%%time
regression = cuml.LinearRegression()
#regression = cuml.Ridge()
x_train, x_test, y_train, y_test  = cuml.train_test_split(listings[columns_to_fit], listings['price'], train_size=0.9)
x_test_index = x_test.reset_index(drop=True)
y_test_index = y_test.reset_index(drop=True)
regression.fit(x_train, y_train)

Visualización de coeficientes y valor de intercepción tras el entrenamiento

In [ ]:
coef_map = cudf.DataFrame()
coef_map['key'] = columns_to_fit
coef_map['value'] = regression.coef_
print("Coefficients:")
coef_map

#print("Intercept:")
#print(regression.intercept_)

Visualización de predicciones

In [ ]:
%%time
predictions = regression.predict(x_test_index)
y_results = cudf.DataFrame()
y_results['prediction'] = predictions
y_results['real'] = y_test_index
y_results[0:10]

Métrica de error

In [ ]:
loss = mnsq(y_test_index, predictions)
print(loss)

# Sección 2: Regresión por CPU

## Carga y tratamiento de datos

In [ ]:
%%time
listings_cpu = pd.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df_cpu = pd.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df_cpu, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if(temp_df_cpu['host_total_listings_count'].dtype != 'int32'):
                    temp_df_cpu['host_total_listings_count'] = temp_df_cpu['host_total_listings_count'].fillna("-1").astype('int32')
                if(temp_df_cpu['bathrooms'].dtype != 'int32'):
                    temp_df_cpu['bathrooms'] = temp_df_cpu['bathrooms'].fillna("-1").astype('int32')
                if(temp_df_cpu['bedrooms'].dtype != 'int32'):
                    temp_df_cpu['bedrooms'] = temp_df_cpu['bedrooms'].fillna("-1").astype('int32')
                if(temp_df_cpu['beds'].dtype != 'int32'):
                    temp_df_cpu['beds'] = temp_df_cpu['beds'].fillna("-1").astype('int32')
                if listings_cpu.size == 0:
                    listings_cpu = temp_df_cpu
                else:
                    for column in listings_cpu.columns:
                        if listings_cpu[column].dtype != temp_df_cpu[column].dtype:
                            print('Found error: '+column+' type '+listings_cpu[column].dtype.name+' doesnt match '+temp_df_cpu[column].dtype.name)
                    listings_cpu = listings_cpu.append(temp_df_cpu)
                    
listings_cpu = listings_cpu.drop_duplicates().reset_index(drop=True)

type_conversion(listings_cpu, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month', 'minimum_nights', 'maximum_nights', 'availability_30', 'availability_90', 'availability_365', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'])
column_factorize(listings_cpu, ['neighbourhood_cleansed', 'host_response_time', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'property_type', 'room_type', 'instant_bookable'])

clean_format_strings(listings_cpu, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price_cpu(listings_cpu, ['price'])

numpy_lat = listings_cpu['latitude'].to_numpy()
numpy_long = listings_cpu['longitude'].to_numpy()
n_numpy_array, e_numpy_array = latlong2osgbgrid_numpy(numpy_lat, numpy_long)
listings_cpu['northing'] = pd.Series(n_numpy_array).astype('float32')
listings_cpu['easting'] = pd.Series(e_numpy_array).astype('float32')
listings_cpu.head()

## Aplicación de modelo de regresión

In [ ]:
%%time
linreg_cpu = LinearRegression()
x_train_cpu, x_test_cpu, y_train_cpu, y_test_cpu  = train_test_split(listings_cpu[['northing', 'easting']], listings_cpu['price'], train_size=0.9)
x_test_cpu_index = x_test_cpu.reset_index(drop=True)
y_test_cpu_index = y_test_cpu.reset_index(drop=True)
linreg_cpu.fit(x_train_cpu, y_train_cpu)

In [ ]:
print("Coefficients: [easting, northing]")
print([linreg_cpu.coef_[0], linreg_cpu.coef_[1]])

print("Intercept:")
print(linreg_cpu.intercept_)

In [ ]:
%%time
predictions_cpu = linreg_cpu.predict(x_test_cpu_index)
y_results_cpu = pd.DataFrame()
y_results_cpu['prediction'] = predictions_cpu
y_results_cpu['real'] = y_test_cpu_index
predictions_cpu[0:10]

In [ ]:
loss_cpu = mnsq_cpu(y_test_cpu_index, predictions_cpu)
print(loss_cpu)